In [ ]:
def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        a, b = eval(js_to_python(s))
        return a
    except Exception as e:
        print(e)
        return []

In [ ]:
def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

In [ ]:
import os
import more_itertools as it2

def get_all_stimuli(trials):
    return [os.path.basename(trial[s]) for trial in trials for s in ['left', 'right']]

def get_group(fn):
    for g in ['low', 'high', 'medium']:
        if g in fn:
            return g

In [ ]:
fn = "../data/experiment/jatos_results_20230102094455.txt"  # single vp export
fn = "../data/experiment/jatos_results_20230102094507.txt"  # export all
with open(fn) as fh:
    raw_data = [read_line(s) for s in fh]
len(raw_data)

In [ ]:
[len(s) for s in raw_data]

In [ ]:
trials = get_trial_data(raw_data[1])

In [ ]:
stimulus_set = set(get_all_stimuli(trials))
stimulus_set

grouped_stimuli = {g: list(it2.bucket(stimulus_set, lambda x: get_group(x))[g]) for g in ['low', 'medium', 'high']}
# grouped_stimuli['low']

In [ ]:
losswin_counts = {}
for g in ['low', 'high', 'medium']:
    losswin_counts[g] = {}
    for i in range(len(grouped_stimuli[g])):
        p1 = grouped_stimuli[g][i]
        # for p2 in grouped_stimuli[g][(i):]:
        for p2 in grouped_stimuli[g][(i+1):]:
            # print(p1, p2)
            losswin_counts[g][(p1, p2)] = [0, 0]

In [ ]:
for trial in trials:
    left, right = [os.path.basename(trial[lr]) for lr in ['left', 'right']]
    if left == right:
        continue
    g = get_group(left)
    idx =  0 if trial['response'] == 'f' else 1
    try:
        losswin_counts[g][(left, right)][idx] += 1
    except KeyError:
        losswin_counts[g][(right, left)][idx-1] += 1
    
# losswin_counts

In [ ]:
# losswin_counts

In [ ]:
import pandas as pd
df = pd.DataFrame(losswin_counts['low']).T
df = df.rename({0: "win1", 1: "win2"}, axis=1)
df

In [ ]:
# df[0] / (df[0] + df[1])

In [ ]:
df.to_pickle("exp_processed.pkl")